# Imviz Demonstration Notebook


This notebook demonstrates the Imviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astropy.utils.data import download_file
from photutils.aperture import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

We also need this to display Matplotlib in the notebook later.

In [ ]:
%matplotlib inline

We create an Imviz instance and grab the default viewer instance as well.

In [ ]:
imviz = Imviz()
viewer = imviz.default_viewer

Download some data. In this example, we use two 47 Tuc observations from HST/ACS.

In [ ]:
acs_47tuc_1 = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/jbqf03gjq_flc.fits', cache=True)
acs_47tuc_2 = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/jbqf03h1q_flc.fits', cache=True)

Load the data into Imviz. Data are linked by pixels as they load. We ignore some warnings that appear during parsing.

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    imviz.load_data(acs_47tuc_1, data_label='acs_47tuc_1')
    imviz.load_data(acs_47tuc_2, data_label='acs_47tuc_2')

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.show()

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
viewer.stretch_options

In [ ]:
viewer.stretch = 'sqrt'

the colormap:

In [ ]:
viewer.colormap_options

In [ ]:
viewer.set_colormap('Viridis')

the limits via the autocut using percentile option:

In [ ]:
viewer.autocut_options

In [ ]:
viewer.cuts = '95%'
viewer.cuts

or the limits directly:

In [ ]:
viewer.cuts = (0, 1000)

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.get_interactive_regions()

In [ ]:
regions

Since the region is an Astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
data = imviz.get_data('acs_47tuc_1[SCI,1]')

In [ ]:
plt.imshow(mask.to_image(data.shape), origin='lower')

It is also possible to programmatically pass a `regions` shape, a `photutils` aperture shape, or a Numpy mask into Imviz.

In [ ]:
c = SkyCoord('00h24m07.33s -71d52m50.71s')

# photutils aperture
my_aper = CircularAperture((600, 400), r=10)
my_aper_sky = SkyCircularAperture(c, 1 * u.arcsec)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=600, y=200), radius=20)
my_reg_sky = CircleSkyRegion(c, Angle(2, u.arcsec))

# Numpy mask
idx = (np.array([350, 350, 350, 350, 350, 350, 351, 351, 351, 351, 352, 352, 352,
                 352, 352, 352, 352, 352, 352, 352, 353, 353, 353, 353, 353, 353,
                 353, 353, 353, 353, 353, 353, 354, 354, 354, 354, 354, 354, 354,
                 354, 355, 355, 355, 355, 355, 355, 355, 355, 356, 356, 356, 356,
                 356, 356, 356, 357, 357, 358, 358]),
       np.array([353, 354, 355, 356, 357, 358, 350, 352, 359, 361, 350, 352, 353,
                 354, 355, 356, 357, 358, 359, 361, 350, 351, 352, 353, 354, 355,
                 356, 357, 358, 359, 360, 361, 351, 352, 354, 355, 356, 357, 359,
                 360, 352, 353, 354, 355, 356, 357, 358, 359, 352, 353, 354, 355,
                 356, 357, 358, 353, 358, 352, 359]))
my_mask = np.zeros(data.shape, dtype=np.bool_)
my_mask[idx] = True

my_regions = [my_aper, my_aper_sky, my_reg, my_reg_sky, my_mask]
imviz.load_regions(my_regions)

You can also programmatically control the viewer.

In [ ]:
# Center the image on given pixel position.
viewer.center_on((1173, 1013))  # X, Y (0-indexed)

In [ ]:
# Move the image with the given pixel offsets.
viewer.offset_by(500, -100)  # dX, dY

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord('00h24m07.33s -71d52m50.71s')
viewer.center_on(sky)

In [ ]:
# Move the image with the given sky offsets.
viewer.offset_by(0.5 * u.arcsec, -1.5 * u.arcsec)

You can programmatically zoom in and out.

Zoom level:

* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display.

In [ ]:
# Get the current zoom level.
viewer.zoom_level

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 1

In [ ]:
# Set the relative zoom based on current zoom level.
viewer.zoom(2)

It is also possible to programmatically add non-interactive markers to the image.

In [ ]:
# Add 100 randomly generated X,Y (0-indexed) w.r.t. reference image
# using default marker properties.
t_xy = Table({'x': np.random.randint(0, 4096, 100),
              'y': np.random.randint(0, 2048, 100)})
viewer.add_markers(t_xy)

You could customize marker color, alpha, size, and fill with values that Glue supports.

In [ ]:
viewer.marker = {'color': 'green', 'alpha': 0.8, 'markersize': 10, 'fill': False}

In [ ]:
# Mark some sky coordinates using updated marker properties.
t_sky = Table({'coord': [SkyCoord('00h24m07.33s -71d52m50.71s'),
                         SkyCoord('00h24m01.57s -71d53m17.77s'),
                         SkyCoord('00h24m11.70s -71d52m29.21s'),
                         SkyCoord('00h24m22.29s -71d53m28.04s')]})
viewer.add_markers(t_sky, use_skycoord=True, marker_name='my_sky')
t_sky

When you do not need the markers anymore, you could remove them by name.

In [ ]:
viewer.remove_markers(marker_name='my_sky')

You can also remove all the markers.

In [ ]:
viewer.reset_markers()

You can save the active display as PNG file.

In [ ]:
viewer.save('myimage.png')

If you want to align the images by WCS instead of pixels, you can run this but it will remove all markers.

In [ ]:
imviz.link_data(link_type='wcs')

If you want a second viewer, run the following cell *or* click on the "picture with +" icon on top of the image viewer.

If you click the icon, the viewer name will be auto-generated. Instead of running the next cell, you can run this instead:

    viewer_2_name = 'imviz-1'
    viewer_2 = imviz.app.get_viewer_by_id(viewer_2_name)

In [ ]:
viewer_2_name = 'my-viewer-1'

# Do not pass in viewer_name if you want it auto-generated.
# If auto-generated, viewer name will be 'imviz-1'.
viewer_2 = imviz.create_image_viewer(viewer_name=viewer_2_name)

You can also programmatically control the new viewer, as follows.

In [ ]:
# Show the first image in the second viewer.
imviz.app.add_data_to_viewer(viewer_2_name, 'acs_47tuc_1[SCI,1]')

In [ ]:
viewer_2.zoom(4)

In [ ]:
viewer_2.cuts = '95%'

In [ ]:
viewer_2.center_on((1173, 1013))

To destroy this new viewer, run the following cell *or* click the "x" that is right next to the viewer ID.

In [ ]:
imviz.destroy_viewer(viewer_2_name)